In [3]:
import pandas as pd

Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [4]:
yelpparsed=pd.read_csv('../data/yelpdata.csv')
foursquareparsed=pd.read_csv('../data/foursqdata.csv')
citybikedata=pd.read_csv('../data/data.csv')

In [9]:
citybikedata.rename(columns={'name': 'station'}, inplace=True)

In [22]:
mergeddf=pd.merge(citybikedata,yelpparsed,on=['station'],how='inner',suffixes=('city','yelp'))
finaldf=pd.merge(mergeddf,foursquareparsed,on=['station'],how='inner',suffixes=('','foursq'))
finaldf=finaldf.rename(columns={'distance': 'distanceyelp','name':'poiname', 'rating':'ratingyelp','popularity':'popularityyelp'})

In [23]:
finaldf.head()

,station,latitude,longitude,free_bikes,empty_slots,total_bike_cap,distanceyelp,poiname,location.address,ratingyelp,distancefoursq,namefoursq,location.addressfoursq,ratingfoursq,popularityyelp,categories
0,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,4.2,773,Union Market,2306 Jefferson Ave,9.4,0.984966,bar
1,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,4.2,912,Liberty Public House,418 N 25th St,8.7,0.975994,bar
2,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,4.2,604,Kahlo’s Taqueria and Bar,718 N 23rd St,7.8,0.971639,bar
3,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,4.2,917,Leisure Business Advisors,2010 Princess Anne Ave,NaN,0.283975,office
4,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,4.2,999,Armstrong Renaissance,1665 N 31st St,NaN,0.324007,office


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [24]:
finaldf['ratingyelp']=finaldf[['ratingyelp']]*2
finaldf.head()

,station,latitude,longitude,free_bikes,empty_slots,total_bike_cap,distanceyelp,poiname,location.address,ratingyelp,distancefoursq,namefoursq,location.addressfoursq,ratingfoursq,popularityyelp,categories
0,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,8.4,773,Union Market,2306 Jefferson Ave,9.4,0.984966,bar
1,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,8.4,912,Liberty Public House,418 N 25th St,8.7,0.975994,bar
2,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,8.4,604,Kahlo’s Taqueria and Bar,718 N 23rd St,7.8,0.971639,bar
3,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,8.4,917,Leisure Business Advisors,2010 Princess Anne Ave,NaN,0.283975,office
4,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,8.4,999,Armstrong Renaissance,1665 N 31st St,NaN,0.324007,office


# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [27]:
import sqlite3
from sqlite3 import Error as e

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection successful")
    except Error as e:
        print(f'The error {e}')
    return connection



connection = create_connection('../data/db')

Connection successful


creating function to execute queries

In [28]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as e:
        print(f"The error '{e}' occurred")

Creating and filling in stations table

In [44]:
create_stations_table = """
CREATE TABLE IF NOT EXISTS stations (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  latitude INTEGER,
  longitude INTEGER,
  free_bikes INTEGER,
  empty_slots INTEGER,
  total_bike_cap INTEGER
); """

execute_query(connection, create_stations_table)  


Query successful


In [45]:
cursor = connection.cursor()
for index, row in citybikedata.iterrows():
    create_station = """
    INSERT INTO 
        stations(name, latitude, longitude, free_bikes, empty_slots, total_bike_cap)
    VALUES
        (?, ?, ?, ?, ?, ?)
    """
    cursor.execute(create_station, (
        row['station'], 
        row['latitude'], 
        row['longitude'], 
        row['free_bikes'], 
        row['empty_slots'], 
        row['total_bike_cap']
    ))
connection.commit()


Creating and inserting yelp table

In [46]:
create_yelp_table = """
CREATE TABLE IF NOT EXISTS yelp (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  station TEXT NOT NULL,
  distance INTEGER,
  name TEXT NOT NULL,
  address TEXT,
  rating DECIMAL
); """

execute_query(connection, create_yelp_table)  


Query successful


In [51]:
for index, row in yelpparsed.iterrows():
    create_yelp = """
    INSERT INTO 
        yelp(station, distance, name, address, rating)
    VALUES
        (?, ?, ?, ?, ?)
    """
    cursor.execute(create_yelp, (
        row['station'], 
        row['distance'], 
        row['name'], 
        row['location.address'], 
        row['rating']
    ))
connection.commit()


Creating and filling Foursquare table

In [52]:
create_foursq_table = """
CREATE TABLE IF NOT EXISTS foursq (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  station TEXT NOT NULL,
  distance INTEGER,
  name TEXT NOT NULL,
  address TEXT,
  rating DECIMAL,
  popularity DECIMAL,
  categories TEXT
); """

execute_query(connection, create_foursq_table)  


Query successful


In [53]:
for index, row in foursquareparsed.iterrows():
    create_foursq = """
    INSERT INTO 
        foursq(station, distance, name, address, rating, popularity, categories)
    VALUES
        (?, ?, ?, ?, ?, ?, ?)
    """
    cursor.execute(create_foursq, (
        row['station'], 
        row['distance'], 
        row['name'], 
        row['location.address'], 
        row['rating'],
        row['popularity'],
        row['categories']
    ))
connection.commit()


Look at the data before and after the join to validate your data.

In [54]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [55]:
query = "SELECT * FROM foursq"
execute_read_query(connection, query)

[(1,
  'East End Library',
  773,
  'Union Market',
  '2306 Jefferson Ave',
  9.4,
  0.9849659425465568,
  'bar'),
 (2,
  'East End Library',
  912,
  'Liberty Public House',
  '418 N 25th St',
  8.7,
  0.975994286012926,
  'bar'),
 (3,
  'East End Library',
  604,
  'Kahlo’s Taqueria and Bar',
  '718 N 23rd St',
  7.8,
  0.971639112938348,
  'bar'),
 (4,
  'East End Library',
  917,
  'Leisure Business Advisors',
  '2010 Princess Anne Ave',
  None,
  0.2839747051547828,
  'office'),
 (5,
  'East End Library',
  999,
  'Armstrong Renaissance',
  '1665 N 31st St',
  None,
  0.3240074560563036,
  'office'),
 (6,
  'Sydney Park',
  295,
  'Beauvine Burger Concept',
  '1501 W Main St',
  8.5,
  0.9884675016985176,
  'bar'),
 (7,
  'Sydney Park',
  170,
  'Postbellum',
  '1323 W Main St',
  7.6,
  0.9516227374875876,
  'bar'),
 (8,
  'Sydney Park',
  828,
  'Cobra Cabana',
  '901 W Marshall St',
  7.9,
  0.995366095848649,
  'bar'),
 (9,
  'Sydney Park',
  974,
  'The Sidewalk Cafe',
  '210

I would execute a join statment to see the same results as a merged dataframe. The query would be as follows

In [ ]:
query = """
SELECT s.station
FROM stations as s
JOIN yelp as y
ON s.station = y.station
"""